In [ ]:
import pickle
import json
import numpy as np
import socket
import time
import numpy as np
from numpy.random import uniform
from numpy.random import choice

PORT = 4424

def move(v, p):
    v = [[vi[0]+p[0], vi[1]+p[1]] for vi in v]
    return v

def resize(v, r):
    v = [[vi[0]*r, vi[1]*r] for vi in v]
    return v

def rot(v, th):
    # x' = xcosθ - ysinθ
    # y' = xsinθ + ycosθ
    x = np.mean(np.array(v)[:,0])
    y = np.mean(np.array(v)[:,1])
    v = move(v, [-int(x), -int(y)])
    v = [[int(vi[0]*np.cos(th) - vi[1]*np.sin(th)), 
          int(vi[0]*np.sin(th) + vi[1]*np.cos(th))] 
         for vi in v]
    v = move(v, [int(x), int(y)])
    return v
    
a = None

'''
int64 -> int
'''
def check_out(v):
    lower_out = np.max(np.array(v)[:,0]) - 479
    upper_out = np.min(np.array(v)[:,0])
    if lower_out > 0:
        v = move(v, [-int(lower_out), 0])
#         v = move(v, [-(480-120), 0])
    elif upper_out < 0:
        v = move(v, [-int(upper_out), 0])
#         v = move(v, [(480-120), 0])
        
    right_out = np.max(np.array(v)[:,1]) - 639
    left_out = np.min(np.array(v)[:,1])
    if right_out > 0:
        v = move(v, [0, -int(right_out)])
#         v = move(v, [0, -(640-120)])
    elif left_out < 0:
        v = move(v, [0, -int(left_out)])
#         v = move(v, [0, (640-120)])
    return v

class Walk(object):
    def __init__(self):
        self.ax, self.ay, self.vx, self.vy = 0., 0., 0., 0.
        self.r_change = uniform(0.1,0.4)
        self.status = choice(['stop', 'walk', 'run'])
        
    def __call__(self, v, d):
        if uniform() < self.r_change:
            self.status = choice(['stop', 'walk', 'run'])
        
        if self.status == 'stop':
            
            self.vx = 0.
            self.ax = 0.
            self.vy = 0.
            self.ay = 0.
            
        elif self.status == 'walk':
            self.vx = d*5.
            self.vy = d*5.
            self.ax = 0.
            self.ay = 0.
        
        elif self.status == 'run':
            self.ax = d*5.
            self.ay = d*5.
        
        self.vx += self.ax
        self.vy += self.ay
        v = move(v,[int(self.vx), int(self.vy)])
        
        return v

class Deform(object): # しぼむ
    pass

class Turn(object):
    def __init__(self):
        self.r_turn = uniform(0.,0.3)
        self.r_mood = uniform(0.,0.1)
        
    def __call__(self, v, d):
        if uniform() > self.r_mood:
            self.r_turn = uniform(0.,0.3)
        if uniform() > self.r_turn:
            return v, d
        x = np.mean(np.array(v)[:,0])
        y = np.mean(np.array(v)[:,1])
        v = move(v, [-int(x), -int(y)])
        v = [[vi[0], -vi[1]] for vi in v]
        v = move(v, [int(x), int(y)])
        d = -d
        return v, d
    
class Jump(object):
    def __init__(self):
        self.r_jump = uniform(0.,0.5)
        self.r_mood = uniform(0.,0.1)
        self.h = uniform(100,200)
        
    def __call__(self, v):
        if uniform() > self.r_mood:
            self.r_jump = uniform(0.,0.5)
        if uniform() > self.r_jump:
            return v
        v = move(v, [-int(uniform(self.h)),0])
        return v
    
class Dance(object):
    def __init__(self):
        self.r_dance = uniform(0.,0.3)
        self.r_mood = uniform(0.,0.1)
        self.is_dancing = False
        self.v0 = None
        self.theta = 0.
        self.d = None
        
    def __call__(self, v):
        if uniform() > self.r_mood:
            self.r_dance = uniform(0.,0.3)
        # not dancing && don't start dance
        if not self.is_dancing and uniform() > self.r_dance:
            return v
        
        x = np.mean(np.array(v)[:,0])
        y = np.mean(np.array(v)[:,1])
        
        if not self.is_dancing:
            self.is_dancing = True
            self.v0 = move(v, [-int(x), -int(y)])
            self.d = choice([-1,1]) # 'right', 'left'
            return v
        
        elif self.theta > 3.1:
            self.is_dancing = False
            self.theta = 0.
            v = move(self.v0, [int(x), int(y)])
            return v
        
        # dance
        else:
            th = np.pi/3.
            v = rot(v, self.d*th)
            self.theta += th
            return v

def gravity(v):
    if np.random.uniform():
        v = move(v, [50,0])
    return v

class Agent(object):
    def __init__(self, name, v):
        self.v0 = v
        self.V = [v,v,v] # t,t-1,t-2
        self.name = name
        self.walk = Walk()
        self.turn = Turn()
        self.jump = Jump()
        self.dance = Dance()
        self.d = 1
        
        self.post()
    def update(self):
        # 参照渡しに注意。
        v = self.V[0]
        d = self.d
        v, d = self.turn(v, d)
        v = self.walk(v, d)
        v = self.jump(v)
        v = self.dance(v)
        v = gravity(v)
        v = check_out(v)
        self.V = [v, self.V[0], self.V[1]]
        self.d = d
        try:
            self.post()
        except:
            pass
    def post(self):
        data = {"queue": "post", 
                "data": {"name": self.name, "vertices": self.V[0]}}
        data = json.dumps(data).encode()
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect(('127.0.0.1', PORT))
            s.sendall(data)
            data = s.recv(65536)

In [ ]:
agents = {}
for i in range(4):
    with open('data/sample_contour/mnist' + str(i) + '.pkl', mode='rb') as f:
        v = pickle.load(f)
        v = resize(v,4)
        p = [np.random.randint(480-120), np.random.randint(640-120)]
        v = move(v,p)
    agents[i] = Agent("mnist"+str(i), v)

In [ ]:
for t in range(10000):
    [agent.update() for agent in agents.values()]
    time.sleep(0.1)